<a href="https://colab.research.google.com/github/daichira-gif/daichira/blob/main/%E5%9B%BD%E4%BC%9A%E8%AD%B0%E4%BA%8B%E9%8C%B2%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89ver_2_0_1%EF%BC%88%E5%B9%B4%E3%81%94%E3%81%A8%E6%8A%BD%E5%87%BA%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install beautifulsoup4

In [2]:
!pip install requests

## 議事録の著作権について

- 電子計算機による情報解析については、著作権法の定めに従い、著作権者の許諾を得ることなく利用可能
- ソース https://kokkai.ndl.go.jp/help.html#:~:text=%E5%80%8B%E3%80%85%E3%81%AE%E7%99%BA%E8%A8%80%E3%81%AE%E3%81%86%E3%81%A1,%E8%A8%B1%E8%AB%BE%E3%81%8C%E5%BF%85%E8%A6%81%E3%81%A8%E3%81%AA%E3%82%8A%E3%81%BE%E3%81%99%E3%80%82

### 参考URL
https://qiita.com/8_hisakichi_8/items/d6894803d2ebabf3d33b


## 国会図書館APIから議事録を抽出

議事録APIのパラメータ<br>
https://kokkai.ndl.go.jp/api.html#specification

## 年別に抽出,JSON形式版

In [3]:
from google.colab import files

from google.colab import drive
drive.mount('/content/drive')

# 保存用ディレクトリの指定
submit_dir = "/content/drive/MyDrive/gijiroku/"

Mounted at /content/drive


In [ ]:
import json
import time
from urllib.request import urlopen, Request
from urllib.parse import quote
from urllib.error import URLError, HTTPError
import xml.etree.ElementTree as ET
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')
import os
from datetime import datetime, timedelta
import re

def create_query( nameOfHouse, session_date):
    params = {
#        'nameOfMeeting': meeting_name,
        'maximumRecords': 10,
        'nameOfHouse': nameOfHouse,
#        "sessionFrom": session,
#        "sessionTo": session,
#        "issueFrom": issue,
#        "issueTo": issue,
        "from": session_date,
        "until": session_date,
    }
    return '&'.join(['{}={}'.format(key, value) for key, value in params.items()])

def parse_xml(res_xml):
    root = ET.fromstring(res_xml)
    speech_list = []
    try:
        for record in root.findall('./records/record/recordData/meetingRecord'):
            session = record.find('session').text
            nameOfMeeting = record.find('nameOfMeeting').text
            issue = record.find('issue').text
            date = record.find('date').text
            print(nameOfMeeting, issue, date)
            i = 1
            chairman = None
            for speechRecord in record.findall('speechRecord'):
                speaker_group = speechRecord.find("speakerGroup").text
                speaker = speechRecord.find('speaker').text
                contents = speechRecord.find('speech').text
                if not any([speaker is None, speaker_group is None]):
                    sep = "　"
                    contents = sep.join(contents.split(sep)[1:])
                    # Remove the strings that match the pattern '午前●●分散会' and '午後●●分散会'
                    contents = re.sub(r'午[前後].*分散会', '', contents)
                    speech = {}
                    speech["発言No."] = i
                    speech["発言者"] = speaker
                    speech["発言内容"] = contents.replace('\n', '').replace('-', '') .replace('◇', '') .replace('君　　　', '君、')   # Remove newline characters and other
                    speech["国会会期"] = session
                    speech["会議名"] = nameOfMeeting
                    speech["会議号"] = issue
                    speech["会議日付"] = date
                    speech_list.append(speech)
                    i += 1
    except ET.ParseError as e:
        print('ParseError: {}'.format(e.code))
    return speech_list

def process_speech_list(speech_list):
    # 発言者ごとに発言内容をまとめて表示
    for speech in speech_list:
        print(f"発言No.: {speech['発言No.']}")
        print(f"発言者: {speech['発言者']}")
        print(f"発言内容: {speech['発言内容']}")
        print("-" * 30)

def main():

    nameOfHouse = '衆議院'
#    nameOfHouse = '参議院'
#    nameOfHouse = '両院協議会'

    # 取得した国会発言をmain外で色々処理をするためグローバル変数で宣言
    global minutes

    # Define the start and end years
    start_year = 2024
    end_year = 2024

    # Loop over each year
    for year in range(start_year, end_year + 1):
        # Define the start and end dates for the current year
        start_date = datetime(year, 1, 1)
        end_date = datetime(year, 12, 31)

        # Calculate the number of days in the current year
        num_days = (end_date - start_date).days + 1

        # Initialize the minutes list for the current year
        minutes = []

        # Loop over each day in the current year
        for i in range(num_days):
            # Calculate the current session date
            session_date = start_date + timedelta(days=i)

            while True:
                query = create_query( nameOfHouse, session_date)

                # ここでAPIリクエストを送信して議事録を取得する処理を実装する
                # クエリはパーセントエンコードしておく
                request_url = 'http://kokkai.ndl.go.jp/api/1.0/meeting?' + quote(create_query( nameOfHouse= nameOfHouse, session_date=session_date.strftime('%Y-%m-%d')))
 #               print(request_url)
                req = Request(request_url)
                try:
                    # リクエストクエリのURLからオブジェクト（今回はXML）を取得しutf8にデコード
                    with urlopen(req) as res:
                        res_xml = res.read().decode('utf8')

                except HTTPError as e:
                    print('HTTPError: {}'.format(e.reason))
                    break
                except URLError as e:
                    print('URLError: {}'.format(e.reason))
                    break

                # try正常終了時の処理記述
                else:
                    # 取得したxmlを国会発言として扱いやすい形（辞書のリスト）に変換
                    minutes_issue = parse_xml(res_xml)

                    # レスポンスが無い場合にループを抜け出す
                    if not minutes_issue:
                        break
                    else:
                      # 取得したminutesをリストに追加
                      minutes.extend(minutes_issue)

                      # ここでminutesを適切に処理する
        #              process_speech_list(minutes_issue)

                      time.sleep(3)  # インターバルを3秒に設定
                      break
        # ループが終了したら、全てのminutesをJSONLファイルに書き出す
        file_name = f"{nameOfHouse}_{year}.jsonl"
        with open(os.path.join(submit_dir, file_name), "a") as jsonl_file:
            for minute in minutes:
                json.dump(minute, jsonl_file, ensure_ascii=False)
                jsonl_file.write('\n')

# - - - - - - - - - - - - - - - - - -
if __name__ == '__main__':
    main()


議院運営委員会 第14号 2024-01-24
議院運営委員会庶務小委員会 第1号 2024-01-24
議院運営委員会図書館運営小委員会 第1号 2024-01-24
本会議 第1号 2024-01-26
沖縄及び北方問題に関する特別委員会 第1号 2024-01-26
北朝鮮による拉致問題等に関する特別委員会 第1号 2024-01-26
議院運営委員会 第1号 2024-01-26
原子力問題調査特別委員会 第1号 2024-01-26
災害対策特別委員会 第1号 2024-01-26
消費者問題に関する特別委員会 第1号 2024-01-26
情報監視審査会 第1号 2024-01-26
政治倫理の確立及び公職選挙法改正に関する特別委員会 第1号 2024-01-26
地域活性化・こども政策・デジタル社会形成に関する特別委員会 第1号 2024-01-26
議院運営委員会 第2号 2024-01-30
議院運営委員会 第3号 2024-01-31
議院運営委員会 第4号 2024-02-01
懲罰委員会 第1号 2024-02-01
財務金融委員会 第1号 2024-02-13
総務委員会 第1号 2024-02-13
内閣委員会 第1号 2024-02-14
財務金融委員会 第2号 2024-02-16
内閣委員会 第2号 2024-02-16
財務金融委員会 第3号 2024-02-20
財務金融委員会 第4号 2024-02-27


### 以下、ボツコード